In [1]:
import pandas as pd
import os

In [2]:
basepath = 'C:/sentiments/aclImdb/'

In [3]:
labels = {'pos':1,'neg':0}

In [4]:
df = pd.DataFrame()

In [5]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path = os.path.join(basepath,s,l)
        #print(path)
        for fil in os.listdir(path):
            #print(fil)
        with open(os.path.join(path,fil),'rb') as f:
                txt = f.read()
             df = df.append([[txt,labels[l]]],ignore_index=True)
            
df.columns=[str('review'),'sentiment']

In [6]:
df.head()

,review,sentiment
0,"b""I went and saw this movie last night after b...",1
1,b'Actor turned director Bill Paxton follows up...,1
2,b'As a recreational golfer with some knowledge...,1
3,"b""I saw this film in a sneak preview, and it i...",1
4,b'Bill Paxton has taken the true story of the ...,1


In [7]:
df.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [8]:
import numpy as np

In [9]:
from sklearn.utils import shuffle

In [10]:
df = shuffle(df)

In [11]:
df.head()

,review,sentiment
35665,"b""Dark comedy? Gallows humor? How does one mak...",1
33879,"b'Gene Tierney and Dana Andrews, who were both...",1
29641,"b""EDDIE MURPHY DELIRIOUS is easily the funnies...",1
49459,"b'Please, spare me of these movies that teach ...",0
28239,b'This is a film about 17th Century Italian ar...,1


In [12]:
df.to_csv('movie_reviews.csv',index=False)

In [13]:
df_new = pd.read_csv('movie_reviews.csv')

In [14]:
from sklearn.utils import shuffle
df_new = shuffle(df_new)

In [15]:
df_new.head()

,review,sentiment
48598,"b""Where the hell are all these uncharted islan...",0
42314,b'That song keeps humming in my head. Not the ...,0
15482,"b""For a film with so much promise it was disap...",0
21991,b'It was disgusting and painful. What a waste ...,0
42109,"b""You know Jason, you know Freddy, and you kno...",0


In [16]:
df_new.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
#df_new.loc[10,'review']
#[df_new.review.str.contains('<br /><br />')]

\W =matches any non-alphanumeric character;

\D = matches any non-digit character

<[^>]*> :all tag  ex..<br />, <a>
    
[^>] :except '>'

In [17]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)',text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [18]:
    

# \W =matches any non-alphanumeric character;
# \D = matches any non-digit character
#<[^>]*> :all tag  ex..<br />, <a>
# [^>] :except '>' 

In [19]:
preprocessor("</a>This is# a ;test ::-)!</a>")

'this is a test :'

In [20]:
df_new['review'] = df_new['review'].apply(preprocessor)

In [21]:
df_new.head()
#df_new[df_new.review.str.contains('<br /><br />')]

,review,sentiment
48598,b where the hell are all these uncharted islan...,0
42314,b that song keeps humming in my head not the g...,0
15482,b for a film with so much promise it was disap...,0
21991,b it was disgusting and painful what a waste o...,0
42109,b you know jason you know freddy and you know ...,0


In [22]:
df_new.index=range(50000)

In [23]:
df_new.head()

,review,sentiment
0,b where the hell are all these uncharted islan...,0
1,b that song keeps humming in my head not the g...,0
2,b for a film with so much promise it was disap...,0
3,b it was disgusting and painful what a waste o...,0
4,b you know jason you know freddy and you know ...,0


In [24]:
X_train = df_new.loc[:2500, 'review'].values
y_train = df_new.loc[:2500, 'sentiment'].values
X_test = df_new.loc[2500:5000, 'review'].values
y_test = df_new.loc[2500:5000, 'sentiment'].values
y_test

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [25]:
print ( np.bincount(y_test))
print (np.unique(y_test))

[1241 1260]
[0 1]


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import GridSearchCV     #hyper-parameter tunning
from sklearn.model_selection import cross_val_score  #avg testing score

In [27]:
tfidf = TfidfVectorizer(stop_words='english')
#X_new=tfidf.fit_transform(X_train)

#log =LogisticRegression()
#log.fit(X_new,y_train)
param_grid = {'clf__C': [1.0, 10.0, 100.0]}

In [28]:
X_train.ndim

1

In [29]:
lr_tfidf = Pipeline([('vect', tfidf),
                     
                     ('clf', LogisticRegression())])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy')

#lr_tfidf.steps

In [30]:
..gs_lr_tfidf.fit(X_train, y_train)
#print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
#print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__C': [1.0, 10.0, 100.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=0)

In [31]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0} 
CV Accuracy: 0.848


In [32]:
clf = gs_lr_tfidf.best_estimator_

In [33]:
clf.score(X_test,y_test)

0.8532586965213914

In [35]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [36]:
nb = Pipeline([('vect', tfidf),
               ('clf', MultinomialNB())])


In [37]:
nb.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [38]:
nb.score(X_train,y_train)

0.9784086365453818

In [40]:
x=nb.score(X_test,y_test)

In [41]:
star=''
if x>.80:
    star='***'
elif x>.6 and x<.8:
    star='**'
elif x>.5 and x<.6:
    star='*'
else:
    star='1/2'
print('Star Rating of this Movie ',star)

Star Rating of this Movie  ***


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
#cv = CountVectorizer(stop_words='english')
cv = TfidfVectorizer(stop_words='english')

In [ ]:
new_data = cv.fit_transform(X_train)
new_test = cv.transform(X_test)

In [ ]:
new_data.shape

In [ ]:
nb = MultinomialNB()
nb.fit(new_data,y_train)

In [ ]:
nb.score(new_test,y_test)

In [ ]:
nb.score(new_data,y_train)